In [ ]:
import nltk, os, subprocess, code, glob, re, traceback, sys, inspect
# from time import clock, sleep
from tika import parser
import re

# filename = input('Enter the path to the resume: ')

filename = 'resume3.pdf'

print(filename)

parsed_pdf = parser.from_file(filename)

data = parsed_pdf['content']

res = re.sub('\s\s+', ' ', data) # remove extra space

data1 = res
try:
    data1 = str(res)
except:
    print('Error')
    
print('**********************************\n')
print('Raw Data')
print(data1)
print('**********************************\n\n\n')

file1 = open("rawdata.txt","w",encoding="utf-8")
file1.write(data1)
file1.close()

data = ''
try:
    data = str(data.strip())
except:
    data = data.strip()

p = ""
f = False

for ch in data:
    if ch == '\n':
        f = True
    else:
        if f == True:
            f = False
            p = p + '\n'
        p = p + ch

data = ''
try:
    data = str(p)
except:
    data = p

p = ""
f = False

for ch in data:
    if ch == ' ':
        f = True
    else:
        if f == True:
            f = False
            p = p + ' '
        p = p + ch

data = ''
try:
    data = str(p)
except:
    data = p

        
file1 = open("data.txt","w",encoding="utf-8")
file1.write(data)
file1.close()
print(data)

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

def convertPDFToText(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    string = retstr.getvalue()
    retstr.close()
    return string

def preprocess(document):
    '''
    Information Extraction: Preprocess a document with the necessary POS tagging.
    Returns three lists, one with tokens, one with POS tagged lines, one with POS tagged sentences.
    Modules required: nltk
    '''
    try:
        # Try to get rid of special characters
        try:
            document = document.decode('ascii', 'ignore')
        except:
            document = document.encode('ascii', 'ignore')
        # Newlines are one element of structure in the data
        # Helps limit the context and breaks up the data as is intended in resumes - i.e., into points
        lines = [el.strip() for el in document.split("\n") if len(el) > 0]  # Splitting on the basis of newlines 
        # lines = [nltk.word_tokenize(el) for el in lines]    # Tokenize the individual lines
        
        #POS tagging
        # lines = [nltk.pos_tag(el) for el in lines]  # Tag them
        
        # Below approach is slightly different because it splits sentences not just on the basis of newlines, but also full stops 
        # - (barring abbreviations etc.)
        # But it fails miserably at predicting names, so currently using it only for tokenization of the whole document
        sentences = nltk.sent_tokenize(document)    # Split/Tokenize into sentences (List of strings)
        tokens = [nltk.word_tokenize(sent) for sent in sentences]    # Split/Tokenize sentences into words (List of lists of strings)
        
        
        #POS tagging
        # sentences = [nltk.pos_tag(sent) for sent in sentences]    # Tag the tokens - list of lists of tuples - each tuple is (<word>, <tag>)
        
        
        # Next 4 lines convert tokens from a list of list of strings to a list of strings; basically stitches them together
        dummy = []
        for el in tokens:
            dummy += el
        tokens = dummy
        # tokens - words extracted from the doc, lines - split only based on newlines (may have more than one sentence)
        # sentences - split on the basis of rules of grammar
        return tokens, lines, sentences
    except Exception as e:
        print(e)

filename = 'resume3.pdf'
data = convertPDFToText(filename)
file1 = open("data_miner.txt","w",encoding="utf-8")
file1.write(data)
file1.close()

#open text file in read mode
text_file = open("data_miner.txt", "rb")

#read whole file to a string
data1 = text_file.read()


#close file
text_file.close()

        
# tokenize
try:
    [tokens, lines, sentences] = preprocess(data1)
except Exception as e:
    e

print(data1)

In [ ]:
tokens

In [ ]:
lines

In [ ]:
sentences

In [ ]:
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub(' +', ' ', resumeText)
    return resumeText

In [ ]:
def getEmail(inputString): 
        '''
        Given an input string, returns possible matches for emails. Uses regular expression based matching.
        Needs an input string, a dictionary where values are being stored, and an optional parameter for debugging.
        Modules required: clock from time, code.
        '''

        email = None
        try:
            pattern = re.compile(r'\S*@\S*')
            matches = pattern.findall(inputString) # Gets all email addresses as a list
            email = matches
        except Exception as e:
            print(e)

        return email

In [ ]:
def getPhone(inputString):
        '''
        Given an input string, returns possible matches for phone numbers. Uses regular expression based matching.
        Needs an input string, a dictionary where values are being stored, and an optional parameter for debugging.
        Modules required: clock from time, code.
        '''

        number = None
        try:
            pattern = re.compile(r'([+(]?\d+[)\-]?[ \t\r\f\v]*[(]?\d{2,}[()\-]?[ \t\r\f\v]*\d{2,}[()\-]?[ \t\r\f\v]*\d*[ \t\r\f\v]*\d*[ \t\r\f\v]*)')
                # Understanding the above regex
                # +91 or (91) -> [+(]? \d+ -?
                # Metacharacters have to be escaped with \ outside of character classes; inside only hyphen has to be escaped
                # hyphen has to be escaped inside the character class if you're not incidication a range
                # General number formats are 123 456 7890 or 12345 67890 or 1234567890 or 123-456-7890, hence 3 or more digits
                # Amendment to above - some also have (0000) 00 00 00 kind of format
                # \s* is any whitespace character - careful, use [ \t\r\f\v]* instead since newlines are trouble
            match = pattern.findall(inputString)
            # match = [re.sub(r'\s', '', el) for el in match]
                # Get rid of random whitespaces - helps with getting rid of 6 digits or fewer (e.g. pin codes) strings
            # substitute the characters we don't want just for the purpose of checking
            match = [re.sub(r'[,.]', '', el) for el in match if len(re.sub(r'[()\-.,\s+]', '', el))>6]
                # Taking care of years, eg. 2001-2004 etc.
            match = [re.sub(r'\D$', '', el).strip() for el in match]
                # $ matches end of string. This takes care of random trailing non-digit characters. \D is non-digit characters
            match = [el for el in match if len(re.sub(r'\D','',el)) <= 15]
                # Remove number strings that are greater than 15 digits
            try:
                for el in list(match):
                    # Create a copy of the list since you're iterating over it
                    if len(el.split('-')) > 3: continue # Year format YYYY-MM-DD
                    for x in el.split("-"):
                        try:
                            # Error catching is necessary because of possibility of stray non-number characters
                            # if int(re.sub(r'\D', '', x.strip())) in range(1900, 2100):
                            if x.strip()[-4:].isdigit():
                                if int(x.strip()[-4:]) in range(1900, 2100):
                                    # Don't combine the two if statements to avoid a type conversion error
                                    match.remove(el)
                        except:
                            pass
            except:
                pass
            number = match
        except:
            pass

        return number

In [ ]:
def getName(data):
    #Define your grammar using regular expressions
    grammar = r"""Chunk: {<NNP><NNP>+}"""
    chunkParser = nltk.RegexpParser(grammar)
    tagged = nltk.pos_tag(nltk.word_tokenize(data))
    chunked = chunkParser.parse(tagged)
#     print(chunked.draw())
    name = '';
    for subtree in chunked.subtrees(filter=lambda t: t.label() == 'Chunk'):
        name = str(subtree[0][0]) + ' ' + str(subtree[1][0])
        break
        
    return name

In [ ]:
# convert bytes datatype to string
data = data1.decode('ISO-8859-1')

print(getEmail(data))
print(getPhone(data))
print(getName(data))



In [ ]:
text_file = open("data_miner.txt", "rb")

data_tika1 = text_file.read()

data_tika = data_tika1.decode('ISO-8859-1')


data_tika = cleanResume(data_tika)

# print(data_tika)

def getExperience(sentence):
    data = "".join([words.lower() for words in sentence])
#     print(data)
    if re.search('experience', data):
        sen_tokenised = nltk.word_tokenize(data)
        tagged = nltk.pos_tag(sen_tokenised)
        entities = nltk.chunk.ne_chunk(tagged)
        experience = False
        for subtree in entities.subtrees():
            for leaf in subtree.leaves():
                if leaf[1] == 'CD':
                    experience = leaf[0]
        return experience
    else:
        return False
    

linesOfResume = [el.strip() for el in data_tika.split("\n") if len(el) > 0]

for line in linesOfResume:
    res = getExperience(line)
    if (res != False):
        print(line + ' -----------> ' + res)